## Feature Extraction

Our data comes in the form of observations done in X-rays over a course of 16 years.
Each observation is of the order of few thousand seconds long (sometimes less, rarely more) of 
uninterrupted data. For each unterrupted time interval in which data was taken, we have **time series**
with a 0.125 second time resolution, taken in four different **energy bands**. 
Additionally, in each observation, the source could have been in one state continuously, but it also 
could have changed state in the middle. GRS 1915+105 is known to switch between two states several times
on occasion before settling into a new state. 

In summary, our data consists of four simultaneous (correlated, but not necessarily identical) time series.
Our task is to extract meaningful **features** for these time series that will capture the behaviour of 
the source and the different states it goes through such that they will be recognizable by a machine learning
algorithm. 

With the exception of a few deep learning methods (which we will not touch here!), we cannot simply put our 
data into a classifier directly, for several reasons:
- in order to capture the behaviour seen by human classifiers accurately, we need to feed long light curves into the classifier; for 1024s time series, that translates to 1024x4x4 samples, at which point dimensionality is a definite problem: *we need to extract features that will summarize the behaviour in as few numbers as possible*
- even if our time series are longer than the typical behaviour of the source, machine learning classifiers are not phase invariant: if I have two time series of a perfectly sinusoidal signal that are out of phase with each other, the classifier will consider them drastically different, because it compares sample by samples (time bin by time bin): *this means we need to extract features that capture the behaviour of the time series in a phase-invariant way*
- there is behaviour in the time series that may be better captures by an alternative representation: *in particular, quasi-periodic features may be better captured in the Fourier domain than in the time domain*
- GRS1915+105 exhibits clear energy-dependent behaviour over time; that is, the energy spectrum of the source changes both within states and between states. *We need to extract features that describe this energy-dependent behaviour*

### A set of features to explore

We will extract features from three domains and will later explore how predictive they are on supervised machine learning tasks. Features will summarize properties of the four simultaneous time series in three domains: the time domain, the Fourier domain, and the energy domain.

Time domain features are:
- mean count rate `fmu`
- variance in the total light curve `fvar`

Fourier domain features are:
- integrated power in 4 bands: 
    - PA: 0.0039-0.031 Hz, 
    - PB: 0.031-0.25 Hz, 
    - PC: 0.25-2.0 Hz and 
    - PD: 2.0-16.0 Hz
- power colours: PC/PA and PB/PD
- frequency at which maximum power is observed


Energy domain features are:
- mean and covariance of hardness ratios in 2 bands (processed data only, NOT Standard 1 data)
    - (2-6keV)/(2-13keV)
    - (9-20kev)/(2-13keV) 



First, we'll need to load the data. This assumes that you've saved a pickle file with a list of $N$ light curves and states, `[[lc1, state1], [lc2, state2], ..., [lc_n, state_n]]`.

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import numpy as np
import glob
import scipy.stats


import cPickle as pickle
import pandas as pd
import sys

import powerspectrum

sys.path.append("/Users/danielahuppenkothen/work/repositories/LightEcho/code/")

from sklearn.preprocessing import StandardScaler
import linearfilter

The following file contains the prepared light curves:

In [3]:
f = open("../../grs1915_125ms_clean.dat")
d_all = pickle.load(f)
f.close()

Let's break down the frequencies of states in these light curves. 

In [4]:
print("Number of light curves:  " + str(len(d_all)))
states = [d[1] for d in d_all]
st = pd.Series(states)
st.value_counts()

Number of light curves:  2829


chi1      100
chi2       68
rho        53
theta      48
chi4       47
beta       39
delta      39
kappa      38
gamma      36
phi        24
mu         17
nu         17
chi3       12
alpha      10
lambda      9
eta         8
omega       6
dtype: int64

You can see that the classes are *fairly* evenly distributed, with $\chi_2$ having the most light curves in their samples, and $\eta$ and $\omega$ being severely undersampled. 

In [5]:
d_labelled = [d for d in d_all if d[1] != None]

In [9]:
print("There are a total number of %i light curves."%len(d_all))
print("%i of them are labelled. "%len(d_labelled))

There are a total number of 2829 light curves.
571 of them are labelled. 


In the next step, we'll pick out training set, validation set and test set.

In [10]:
## total number of light curves
n_lcs = len(d_all)

## Set the seed to I will always pick out the same light curves.
np.random.seed(20160615)

## shuffle list of light curves
np.random.shuffle(d_all)

train_frac = 0.6
validation_frac = 0.2
test_frac = 0.2

## let's pull out light curves for three data sets into different variables.
d_all_train = d_all[:int(train_frac*n_lcs)]
d_all_val = d_all[int(train_frac*n_lcs):int((train_frac + validation_frac)*n_lcs)]
d_all_test = d_all[int((train_frac + validation_frac)*n_lcs):]

## Let's print some information about the three sets.
print("There are %i light curves in the training set."%len(d_all_train))
print("There are %i light curves in the validation set."%len(d_all_val))
print("There are %i light curves in the test set."%len(d_all_test))
for da,n in zip([d_all_train, d_all_val, d_all_test], ["training", "validation", "test"]):
    print("These is the distribution of states in the %s set: "%n)
    states = [d[1] for d in da]
    st = pd.Series(states)
    print(st.value_counts())
    print("================================================================")

There are 1697 light curves in the training set.
There are 566 light curves in the validation set.
There are 566 light curves in the test set.
These is the distribution of states in the training set: 
chi1      59
chi2      37
rho       34
chi4      33
theta     29
delta     26
kappa     25
gamma     23
beta      20
phi       14
mu        12
nu        11
alpha      6
chi3       6
lambda     6
eta        4
omega      3
dtype: int64
These is the distribution of states in the validation set: 
chi1      19
chi2      15
theta     11
chi4       8
beta       7
kappa      7
rho        7
gamma      5
delta      5
phi        4
chi3       3
lambda     3
eta        3
nu         3
omega      2
mu         2
alpha      1
dtype: int64
These is the distribution of states in the test set: 
chi1     22
chi2     16
rho      12
beta     12
gamma     8
theta     8
delta     8
kappa     6
chi4      6
phi       6
nu        3
mu        3
alpha     3
chi3      3
eta       1
omega     1
dtype: int64


As a next step, we can make light curve segments, overlapping or not, to then pass to feature extraction. 

In [41]:

## This function is also in grs1915_utils.py!
def extract_segments(d_all, seg_length = 256., overlap=128., dt=0.125):
    """ Extract light curve segmens from a list of light curves. 
        Each element in the list is a list with two elements: 
        - an array that contains the light curve in three energy bands 
        (full, low energies, high energies) and 
        - a string containing the state of that light curve.
        
        Parameters
        ==========
        seg_length : float
            The length of each segment. Bits of data at the end of a light curve
            that are smaller than seg_length will not be included. 
        
        overlap : float 
            This is actually the interval between start times of individual segments,
            i.e. by default light curves start 64 seconds apart. The actual overlap is 
            seg_length-overlap
            
        dt : float
            Set the time resolution of the light curve by hand. I allow this because 
            in the GRS 1915+105 data, there are rounding errors affecting the time 
            resolution, which in turn affects the length of the segments. 
    """
    segments, labels = [], [] ## labels for labelled data    
    
    for i,d_seg in enumerate(d_all):
        
        ## data is an array in the first element of d_seg
        data = d_seg[0]
        ## state is a string in the second element of d_seg
        state = d_seg[1]
        
        ## if the light curve is shorter than the segment length,
        ## throw out!
        length = data[-1,0] - data[0,0]
        if length < seg_length:
            continue
            
        ## compute the number of time bins in a segment
        nseg = seg_length/dt
        ## compute the number of time bins to start of next segment
        noverlap = overlap/dt
        
        istart = 0
        iend = nseg
        j = 0
     
        while iend <= len(data):
            dtemp = data[istart:iend]
            segments.append(dtemp)
            labels.append(state)
            istart += noverlap
            iend += noverlap
            j+=1
        
    return segments, labels


We'll extract 1024 second long segments for the supervised classification.

For the supervised classification, we would like to capture as much of the states as observed by Belloni et al in  each light curve (such that there isn't too much variance between features of samples within a class), but we also need to retain enough light curves for a useful training set. 

We can choose some overlap between segments to augment our data set and allow the classifier to "see" different parts of the same light curve shape. Here, we choose 128 seconds for all further analysis!

In [42]:
seg_length_supervised = 1024.

overlap_all = 128.
dt = 0.125

## CURRENTLY RUNNING LONG SEGMENTS FOR SUPERVISED CLASSIFICATION
seg_train, labels_train = extract_segments(d_all_train, seg_length=seg_length_supervised, 
                                           overlap=overlap_all, dt=0.125)
seg_val, labels_val = extract_segments(d_all_val, seg_length=seg_length_supervised, 
                                       overlap=overlap_all, dt=0.125)
seg_test, labels_test = extract_segments(d_all_test, seg_length=seg_length_supervised, 
                                         overlap=overlap_all, dt=0.125)

## Let's print some details on the different segment data sets
print("There are %i segments in the training set."%len(seg_train))
print("There are %i segments in the validation set."%len(seg_val))
print("There are %i segments in the test set."%len(seg_test))
for la,n in zip([labels_train, labels_val, labels_test], ["training", "validation", "test"]):
    print("These is the distribution of states in the %s set: "%n)
    st = pd.Series(la)
    print(st.value_counts())
    print("================================================================")

There are 10164 segments in the training set.
There are 3641 segments in the validation set.
There are 3625 segments in the test set.
These is the distribution of states in the training set: 
chi1      320
chi2      262
chi4      205
theta     197
gamma     191
rho       184
phi       153
kappa     126
beta      108
mu        104
delta      68
nu         43
lambda     42
alpha      35
eta        29
chi3       22
omega      10
dtype: int64
These is the distribution of states in the validation set: 
chi1      121
chi2      118
theta      80
chi4       62
beta       50
chi3       42
eta        42
gamma      36
lambda     25
nu         21
omega      18
rho        18
phi        18
mu         18
kappa      17
delta       3
dtype: int64
These is the distribution of states in the test set: 
chi2     138
beta     106
chi1     101
rho       95
theta     84
gamma     65
chi4      36
kappa     36
chi3      35
phi       32
delta     26
alpha     18
eta       17
nu        13
omega      3
dtype: int6

/scratch/daniela/sw/miniconda/lib/python2.7/site-packages/ipykernel/__main__.py:52: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


And now, we can think about actual feature extraction.

Let's make some functions to extract individual features from a segment. We can then combine them at will. 

### Time Series Features

First, we're going to extract time series features derived from the light curve:

- mean, 
- median 
- variance 
- skewness
- kurtosis
- weights from a linear filter 

The linear filter requires two hyperparameters:
1. the number of previous time bins to use in the determination of the current flux; this translates directly into the number of features added (i.e. $k=10$ will translate into 10 features added by this representation of the data)
2. the regularization term for the ridge regression (which can also be determined via cross validation)

We will determine the linear filter hyperparameters in the feature engineering step of the analysis.

**NOTE: The linear filter routine takes *all* light curves at the same time! Additionally, we'll want these 
light curves all scaled to the same mean and variance before applying the filter!**

In [43]:
### extract summary statistics from the time series
def timeseries_features(seg):
    counts = seg[:,1] ## extract counts in full band
    fmean = np.mean(counts)                    ## mean
    fmedian = np.median(counts)                ## median
    fvar = np.var(counts)                      ## variance
    skew = scipy.stats.skew(counts)            ## skewness
    kurt = scipy.stats.kurtosis(counts)        ## kurtosis
    return fmean, fmedian, fvar, skew, kurt


def linear_filter(counts_scaled, k=10):
    """
    Extract features from a linear filter. This is actually an autoregressive model!
    :param counts_scaled: numpy ndarray (nsamples, ntimebins) with SCALED TOTAL COUNTS
    :return ww_all: numpy ndarray (nsamples, k) of weights 
    """
  
    ## initialize the LinearFilter object
    lf = linearfilter.LinearFilter(k=k)
    
    ww_all = np.zeros((len(counts_scaled), k))
    
    ## loop through all light curves and compute the weight vector for each
    for i,c in enumerate(counts_scaled):
        ww_all[i,:] = lf.fit_transform(c)[0]
    return ww_all


## Fourier Domain Features

We'll extract the integrated power in four power spectral bands, as well as power colours (ratios of these integrated PSD bands) and the maximum frequency at which we observe the maximum power (as a proxy for the presence of a QPO). 
The power spectral bands are taken from Lucy's paper.

In [44]:
## boundaries for power spectral bands, from Heil et al, 2014
pcb = {"pa_min":0.0039, "pa_max":0.031,
       "pb_min":0.031, "pb_max":0.25,
       "pc_min":0.25, "pc_max":2.0,
       "pd_min":2.0, "pd_max":16.0}

def rebin_psd(freq, ps, n=10, type='average'):

    nbins = int(len(freq)/n)
    df = freq[1] - freq[0]
    T = freq[-1] - freq[0] + df
    bin_df = df*n
    binfreq = np.arange(nbins)*bin_df + bin_df/2.0 + freq[0]

    #print("len(ps): " + str(len(ps)))
    #print("n: " + str(n))

    nbins_new = int(len(ps)/n)
    ps_new = ps[:nbins_new*n]
    binps = np.reshape(np.array(ps_new), (nbins_new, int(n)))
    binps = np.sum(binps, axis=1)
    if type in ["average", "mean"]:
        binps = binps/np.float(n)
    else:
        binps = binps

    if len(binfreq) < len(binps):
        binps= binps[:len(binfreq)]

    return binfreq, binps


def psd_features(seg, pcb):
    """
    Computer PSD-based features.
    seg: data slice of type [times, count rates, count rate error]^T
    pcb: frequency bands to use for power colours
    """

    times = seg[:,0]
    dt = times[1:] - times[:-1]
    dt = np.min(dt)

    counts = seg[:,1]*dt
    #ps = powerspectrum.PowerSpectrum(times, counts=counts, norm="rms")
    freq, ps = make_psd(seg, navg=1)
    #print("len(ps), before: " + str(len(ps)))
    if times[-1]-times[0] >= 2.*128.0:
        tlen = (times[-1]-times[0])
        nrebin = np.round(tlen/128.)
        freq, ps = rebin_psd(freq, ps, n=nrebin, type='average')

    #print("len(ps), after: " + str(len(ps)))

    freq = np.array(freq[1:])
    ps = ps[1:]
    #print("min(freq): " + str(np.min(freq)))
    #print("max(freq): " + str(np.max(freq)))
    #print("len(freq): " + str(len(freq)))

    binfreq, binps = total_psd(seg, 24)

    #print("min(binps): " + str(np.min(binps)))
    #print("max(binps): " + str(np.max(binps)))
    fmax_ind = np.where(binps == np.max(binps))
    #print("fmax_ind: " + str(fmax_ind))
    maxfreq = binfreq[fmax_ind[0]]
    #print("maxfreq: " + str(maxfreq))

    ## find power in spectral bands for power-colours
    pa_min_freq = freq.searchsorted(pcb["pa_min"])
    pa_max_freq = freq.searchsorted(pcb["pa_max"])

    pb_min_freq = freq.searchsorted(pcb["pb_min"])
    pb_max_freq = freq.searchsorted(pcb["pb_max"])

    pc_min_freq = freq.searchsorted(pcb["pc_min"])
    pc_max_freq = freq.searchsorted(pcb["pc_max"])

    pd_min_freq = freq.searchsorted(pcb["pd_min"])
    pd_max_freq = freq.searchsorted(pcb["pd_max"])

    psd_a = np.sum(ps[pa_min_freq:pa_max_freq])
    psd_b = np.sum(ps[pb_min_freq:pb_max_freq])
    psd_c = np.sum(ps[pc_min_freq:pc_max_freq])
    psd_d = np.sum(ps[pd_min_freq:pd_max_freq])
    pc1 = np.sum(ps[pc_min_freq:pc_max_freq])/np.sum(ps[pa_min_freq:pa_max_freq])
    pc2 = np.sum(ps[pb_min_freq:pb_max_freq])/np.sum(ps[pd_min_freq:pd_max_freq])

    return maxfreq, psd_a, psd_b, psd_c, psd_d, pc1, pc2

def make_psd(segment, navg=1):

    times = segment[:,0]
    dt = times[1:] - times[:-1]
    dt = np.min(dt)

    counts = segment[:,1]*dt

    tseg = times[-1]-times[0]
    nlc = len(times)
    nseg = int(nlc/navg)

    if navg == 1:
        ps = powerspectrum.PowerSpectrum(times, counts=counts, norm="rms")
        ps.freq = np.array(ps.freq)
        ps.ps = np.array(ps.ps)*ps.freq
        return ps.freq, ps.ps
    else:
        ps_all = []
        for n in xrange(navg):
            t_small = times[n*nseg:(n+1)*nseg]
            c_small = counts[n*nseg:(n+1)*nseg]
            ps = powerspectrum.PowerSpectrum(t_small, counts=c_small, norm="rms")
            ps.freq = np.array(ps.freq)
            ps.ps = np.array(ps.ps)*ps.freq
            ps_all.append(ps.ps)

        #print(np.array(ps_all).shape)

        ps_all = np.average(np.array(ps_all), axis=0)

        #print(ps_all.shape)

    return ps.freq, ps_all

epsilon = 1.e-8

def total_psd(seg, bins):
    times = seg[:,0]
    dt = times[1:] - times[:-1]
    dt = np.min(dt)
    counts = seg[:,1]*dt

    ps = powerspectrum.PowerSpectrum(times, counts=counts, norm="rms")
    ps.ps = np.array(ps.freq)*np.array(ps.ps)
    binfreq = np.logspace(np.log10(ps.freq[1]-epsilon), np.log10(ps.freq[-1]+epsilon), bins)
    #print("freq: " + str(ps.freq[1:10]))
    #print("binfreq: " + str(binfreq[:10]))
    binps, bin_edges, binno = scipy.stats.binned_statistic(ps.freq[1:], ps.ps[1:], statistic="mean", bins=binfreq)
    df = binfreq[1:]-binfreq[:-1]
    binfreq = binfreq[:-1]+df/2.

    return np.array(binfreq), np.array(binps)




### Energy Domain Features

Hardness ratios, the ratio of the flux in one photon energy band versus the flux in another, is a typical and useful proxy for more detailed energy spectral modelling. X-ray binaries are well known for their changing energy spectra in different stages of outburst, and GRS 1915+105 is no exception. Any features relating to hardness ratios are useful not only because they are likely to be discriminative, but also because they have a direct link to physical processes in the source, and are directly interpretable to physicists. 

The *Rossi* X-ray Timing Explorer (RXTE for short) covers the energy range between 2 and 70 keV. Our data is split into four bands, one covering the whole range, and three that covers the lower range, midrange and upper range of the energy bands (need numbers here!). Hardness ratios are generally derived by dividing the flux in the midrange and upper range by the flux in the lower energy band. These are called hardness ratio 1 and 2, respectively (need to add exact definitions). 
In general, a scatter plot of one hardness ratio against another (an X-ray colour-colour diagram) for a time series reveals that in different states, hardness ratios can differ by a facter of 10 or more. In some states, the source also makes a certain track in the colour-colour diagram. 

Our task is to summarize this behaviour in as few features as possible. 
The approach we choose here is particularly simple: we compute means of both hardness ratios for each segment, as well as the covariance between the two. This gives us a total of five numbers: two means, two variances (diagonal of the covariance matrix) and one covariance (the other is symmetric to the first).
This will likely **not** capture the full behaviour, because for some states, the colour-colour diagram isn't particularly well modelled by a 2D Gaussian, but our hope is that it will be good enough for the classification tasks attempted here. We also extract skewness and kurtosis for both hardness ratios, which we hope will capture some of the non-Gaussianities in the hardness ratios.

We can also experiment with more complex representations (for example a 2D histogram representation) or a bivariate Gaussian distribution fit, but initial experiments suggest that these representations do not particularly increase recall or precision in classification tasks.

The code is still here for completeness, as is a function that computes a 2D histogram of the hardness-intensity diagram (HID), another diagnostic often used in X-ray binary research, but that does not seem to add anything to classification tasks, either.

In [45]:
def compute_hrlimits(hr1, hr2):
    """
    Limits on the hardness ratios based on all segments.
    Used to generate the ranges for the 2-D histograms.
    """
    
    min_hr1 = np.min(hr1)
    max_hr1 = np.max(hr1)

    min_hr2 = np.min(hr2)
    max_hr2 = np.max(hr2)
    return [[min_hr1, max_hr1], [min_hr2, max_hr2]]

def hr_maps(seg, bins=30, hrlimits=None):
    times = seg[:,0]
    counts = seg[:,1]
    low_counts = seg[:,2]
    mid_counts = seg[:,3]
    high_counts = seg[:,4]
    hr1 = np.log(mid_counts/low_counts)
    hr2 = np.log(high_counts/low_counts)

    if hrlimits is None:
        hr_limits = compute_hrlimits(hr1, hr2)
    else:
        hr_limits = hrlimits

    h, xedges, yedges = np.histogram2d(hr1, hr2, bins=bins,
                                       range=hr_limits)
    h = np.rot90(h)
    h = np.flipud(h)
    hmax = np.max(h)
    #print(hmax)
    hmask = np.where(h > hmax/20.)
    hmask1 = np.where(h < hmax/20.)
    hnew = copy.copy(h)
    hnew[hmask[0], hmask[1]] = 1.
    hnew[hmask1[0], hmask1[1]] = 0.0
    return xedges, yedges, hnew

def hr_fitting(seg):
    counts = seg[:,1]
    low_counts = seg[:,2]
    mid_counts = seg[:,3]
    high_counts = seg[:,4]
    hr1 = mid_counts/low_counts
    hr2 = high_counts/low_counts

    # sometimes, low_counts is zero, which leads to 
    # infinite values in HR1 or HR2, which we will remove
    hr1_mask = np.where(np.isfinite(hr1))
    hr1 = hr1[hr1_mask]
    hr2 = hr2[hr1_mask]

    hr2_mask = np.where(np.isfinite(hr2))
    hr1 = hr1[hr2_mask]
    hr2 = hr2[hr2_mask]

    mu1 = np.mean(hr1)
    mu2 = np.mean(hr2)

    cov = np.cov(hr1, hr2).flatten()

    skew = scipy.stats.skew(np.array([hr1, hr2]).T)
    kurt = scipy.stats.kurtosis(np.array([hr1, hr2]).T)

    if np.any(np.isnan(cov)):
        print("NaN in cov")

    return mu1, mu2, cov, skew, kurt

def hid_maps(seg, bins=30):
    counts = seg[:,1]
    low_counts = seg[:,2]
    high_counts = seg[:,3]
    hr2 = high_counts/low_counts
    hid_limits = compute_hrlimits(hr2, counts)

    h, xedges, yedges = np.histogram2d(hr2, counts, bins=bins,
                                       range=hid_limits)
    h = np.rot90(h)
    h = np.flipud(h)

    return xedges, yedges, h


Below, for completeness, are another few snippets of code that extracts binned versions of the light curve in case I ever want to plug these into a classifier (but I generally don't) as well as a piece of code that extracts the (1) full light curve, (2) hardness ratio 1 and (3) hardness ratio 2. 

In [46]:
def lcshape_features(seg, dt=1.0):
    
    times = seg[:,0]
    counts = seg[:,1]

    dt_small = times[1:]-times[:-1]
    dt_small = np.min(dt_small)

    nbins = np.round(dt/dt_small)

    bintimes, bincounts = rebin_psd(times, counts, nbins)
    
    return bincounts

def extract_lc(seg):
    times = seg[:,0]
    counts = seg[:,1]
    low_counts = seg[:,2]
    high_counts = seg[:,3]
    hr1 = low_counts/counts
    hr2 = high_counts/counts
    return [times, counts, hr1, hr2]


### Making Features

Now we can use this code to extract some features and put them into a giant vector.
We will need to extract the time series features using the linear filter separately, because this involves scaling *all* light curves to unit mean and variance, and then using `LinearFilterEnsemble` on the entire thing. 
So, we'll first extract all other features, then we'll extract the features from the linear filter, and then we'll stack them all together. Easy.

In [49]:
def make_features(seg, k=10, bins=30, navg=4, hr_summary=True, ps_summary=True, lc=True, hr=True, hrlimits=None):
    """
    Make features from a set of light curve segments, except for the linear filter!
    
    :param seg: list of all segments to be used
    :param bins: bins used in a 2D histogram if hr_summary is False
    :param hr_summary: if True, summarize HRs in means and covariance matrix
    :param ps_summary: if True, summarize power spectrum in frequency of maximum power and power spectral bands
    :param lc: if True, store light curves
    :param hr: if True, store hardness ratios
    :param hrlimits: limits for the 2D histogram if hr_summary is False
    :return: fdict: dictionary with keywords "features", "lc" and "hr"
    """
    features = []
    if lc:
        lc_all = []
    if hr:
        hr_all = []
        
        
    ## MAKE FEATURES BASED ON LINEAR FILTER

    ## first, extract the total counts out of each segment
    counts = np.array([s[:,1] for s in seg])

    ## next, transform the array such that *each light curve* is scaled
    ## to zero mean and unit variance
    ## We can do this for all light curves independently, because we're
    ## averaging *per light curve* and not *per time bin*
    counts_scaled = StandardScaler().fit_transform(counts.T).T

    ## transform the counts into a weight vector
    ww = linear_filter(counts_scaled, k=k)
    
    for s in seg:

        features_temp = []

        ## time series summary features
        fmean, fmedian, fvar, fskew, fkurt = timeseries_features(s)
        features_temp.extend([fmean, fmedian, fvar, fskew, fkurt])

        if ps_summary:
            ## PSD summary features
            maxfreq, psd_a, psd_b, psd_c, psd_d, pc1, pc2 = psd_features(s, pcb)
            features_temp.extend([maxfreq, psd_a, psd_b, psd_c, psd_d, pc1, pc2])
        else:
            ## whole PSD
            binfreq, binps = total_psd(s, 24)
            features_temp.extend(binps[1:])

        if hr_summary:
            mu1, mu2, cov, skew, kurt = hr_fitting(s)
            features_temp.extend([mu1, mu2])
            features_temp.extend([cov[0], cov[1], cov[3]])
            features_temp.extend(list(skew))
            features_temp.extend(list(kurt))

        else:
            xedges, yedges, h = hr_maps(s, bins=bins, hrlimits=hrlimits)
            features_temp.extend(h.flatten())

        features.append(features_temp)

        if lc or hr:
            lc_temp = extract_lc(s)
        if lc:
            lc_all.append([lc_temp[0], lc_temp[1]])
        if hr:
            hr_all.append([lc_temp[2], lc_temp[3]])

    features_all = np.hstack((np.array(features), ww))
    
    fdict = {"features": features_all}
    if lc:
        fdict["lc"] = lc_all
    if hr:
        fdict["hr"] = hr_all
    return fdict



Now we can make the dictionaries with features, light curves and hardness ratios:

In [50]:
features_train = make_features(seg_train[:6], bins=30)
features_val = make_features(seg_val[:4], bins=30)
features_test = make_features(seg_test[:5], bins=30)


/scratch/daniela/sw/miniconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


The order of the features is the following:


1. mean
2. median
3. variance
4. skewness
5. kurtosis

If `ps_summary` is set to `True`, then the list continues:

6. frequency where the periodogram has its maximum power
7. integrated power in PSD band A
8. integrated power in PSD band B
9. integrated power in PSD band C
10. integrated power in PSD band D
11. Power Colour 1: PSD C / PSD A
12. Power Colour 2: PSD B / BSD D

otherwise features 6-30 are the power in 24 power spectral bins.

If `hr_summary` is set to `True`, then the list continues:

13. mean of HR 1
14. mean of HR 2
15. variance of HR 1
16. covariance between HR 1 and HR 2 
17. variance of HR 2
18. skewness of HR 1
19. skewness of HR 2
20. kurtosis of HR 1
21. kurtosis of HR 2

otherwise, the next set of features is an `(nbins x nbins)` long list
of bins from a 2D histogram representation.

The last `[-k:]` features contain the `k`-long weight vector for each 
light curve describing the linear filter.


### Saving the data for future use

Let's save our features in a bunch of files for future use. 
Because the feature matrices are not that big, they go into a simple ascii file.
Light curves and hardness ratios go into pickle files. Let's define a little helper
function to make this easier.

In [51]:
## Remember how we're doing the longer segments right now? 
## Yeah, don't forget that in the file name!

def save_features(fdict, seg_length, lc=True, hr=True, froot="../../grs1915"):
    froot = froot + str(int(seg_length))
    np.savetxt(froot + "_features.txt", fdict["features"])
    if lc:
        f = open(froot + "_lightcurves.dat", "w")
        pickle.dump(fdict["lc"], f)
        f.close()
    if hr:
        f = open(froot + "_hardness.dat", "w")
        pickle.dump(fdict["hr"], f)
        f.close()
        
    return

In [52]:
save_features(features_train, seg_length_supervised)

## Combining it all

The code above starts with a file called `grs1915_all_125ms.dat`. This file 
has the data extracted from their original fits files and separated observations
into individual light curves split up by data gaps. 

First, we define a helper function that goes through each observation, finds data gaps and splits the data along these gaps into uneven segments.

In [53]:
import generaltools as gt

def remove_gaps(d, state):
    """
    Split light curves into segments without gaps
    """
    dt_data = d[1:,0]-d[:-1,0]
    dt_min = np.min(dt_data)
    tol = dt_min*0.01
    
    ### split data with breaks
    breaks = np.where(dt_data > dt_min+tol)[0]

    d_all = []
    
    if len(breaks) == 0:
        dtemp = d
        d_all.append([dtemp, state])
    else:
        for i,b in enumerate(breaks):
            if i == 0:
                if b == 0:
                    continue
                else:
                    dtemp = d[:b]

            else:
                dtemp = d[breaks[i-1]+1:b]

            d_all.append([dtemp, state])

        ## last segment
        dtemp = d[b+1:]
        d_all.append([dtemp, state])

    return d_all



We might also need a function that rebins the data into light curves with a coarser 
time resolution:

In [54]:
## helper function that bins light curves
def bin_lightcurve(dtemp, nbins):
    tbinned_times, tbinned_counts = gt.rebin_lightcurve(dtemp[:,0], dtemp[:,1], n=nbins, type="average")
    lbinned_times, lbinned_counts = gt.rebin_lightcurve(dtemp[:,0], dtemp[:,2], n=nbins, type="average")
    hbinned_times, hbinned_counts = gt.rebin_lightcurve(dtemp[:,0], dtemp[:,3], n=nbins, type="average")
    dshort = np.transpose(np.array([tbinned_times, tbinned_counts, lbinned_counts, hbinned_counts]))
    return dshort


Sometimes, for whatever reason, `inf`s and `nan`s might sneak into our feature vectors. 
This will make the Standard Scaler (and various machine learning algorithms) very unhappy,
so we'd like to remove them:

In [55]:
def check_nan(features, labels, hr=True, lc=True):
    inf_ind = []
    fnew, lnew, tnew = [], [], []
    if lc:
        lcnew = []
    if hr:
        hrnew = []

    for i,f in enumerate(features["features"]):

        try:
            if any(np.isnan(f)):
                print("NaN in sample row %i"%i)
                continue
            elif any(np.isinf(f)):
                print("inf sample row %i"%i)
                continue
            else:
                fnew.append(f)
                lnew.append(labels[i])
                tnew.append(features["tstart"][i])
                if lc:
                    lcnew.append(features["lc"][i])
                if hr:
                    hrnew.append(features["hr"][i])
        except ValueError:
            raise Exception("This is breaking! Boo!")
    features_new = {"features":fnew, "tstart":tnew}
    if lc:
        features_new["lc"] = lcnew
    if hr:
        features_new["hr"] = hrnew
    return features_new, lnew


Below is the function that actually does all the feature extraction all in one step. Note that for extracting different features, you'll need to change those in the function below. I might make that more robust in the future.

In [63]:
import convert_belloni 

def make_all_features(d_all, k=10, lamb=0.1, val=True, train_frac=0.6, validation_frac=0.2, test_frac = 0.2,
                  seg=True, seg_length=1024., overlap=64., dt=0.125,
                  bins=30, navg=4, hr_summary=True, ps_summary=True, lc=True, hr=True,
                  save_features=True, froot="grs1915", seed=20160615):

    ## Set the seed to I will always pick out the same light curves.
    np.random.seed(seed)

    ## shuffle list of light curves
    indices = np.arange(len(d_all))
    np.random.shuffle(indices)

    n_lcs = len(d_all)

    ## let's pull out light curves for three data sets into different variables.
    d_all_train = [d_all[i] for i in indices[:int(train_frac*n_lcs)]]
    d_all_test = [d_all[i] for i in indices[int(train_frac*n_lcs):int((train_frac + test_frac)*n_lcs)]]

    seg_train, labels_train = extract_segments(d_all_train, seg_length=seg_length, 
                                               overlap=overlap, dt=dt)
    seg_test, labels_test = extract_segments(d_all_test, seg_length=seg_length, 
                                             overlap=overlap, dt=dt)

    if val:
        d_all_val = [d_all[i] for i in indices[int((train_frac + test_frac)*n_lcs):]]
        seg_val, labels_val = extract_segments(d_all_val, seg_length=seg_length, 
                                               overlap=overlap, dt=dt)


    ### hrlimits are derived from the data, in the GRS1915_DataVisualisation Notebook
    hrlimits = [[-2.5, 1.5], [-3.0, 2.0]]

    features_train = make_features(seg_train,k, bins, lamb, hr_summary, ps_summary, lc, hr, hrlimits=hrlimits)
    features_test = make_features(seg_test,k, bins,  lamb,hr_summary, ps_summary, lc, hr, hrlimits=hrlimits)

    features_train["tstart"] = tstart_train
    features_test["tstart"] = tstart_test

    ## check for NaN
    print("Checking for NaN in the training set ...")
    features_train_checked, labels_train_checked = check_nan(features_train, labels_train, hr=hr, lc=lc)
    print("Checking for NaN in the test set ...")
    features_test_checked, labels_test_checked = check_nan(features_test, labels_test, hr=hr, lc=lc)


    labelled_features = {"train": [features_train_checked["features"], labels_train_checked],
                     "test": [features_test_checked["features"], labels_test_checked]}

    if val:
        features_val = make_features(seg_val, k, bins, lamb, hr_summary, ps_summary, lc, hr, hrlimits=hrlimits)
        features_val["tstart"] = tstart_val
        
        print("Checking for NaN in the validation set ...")
        features_val_checked, labels_val_checked = check_nan(features_val, labels_val, hr=hr, lc=lc)

        labelled_features["val"] =  [features_val_checked["features"], labels_val_checked],

    if save_features:
        np.savetxt(froot+"%i_features_train.txt"%int(seg_length), features_train_checked["features"])
        np.savetxt(froot+"%i_features_test.txt"%int(seg_length), features_test_checked["features"])

        np.savetxt(froot+"%i_tstart_train.txt"%int(seg_length), features_train_checked["tstart"])
        np.savetxt(froot+"%i_tstart_test.txt"%int(seg_length), features_test_checked["tstart"])

        ltrainfile = open(froot+"%i_labels_train.txt"%int(seg_length), "w")
        for l in labels_train_checked:
            ltrainfile.write(str(l) + "\n")
        ltrainfile.close()

        ltestfile = open(froot+"%i_labels_test.txt"%int(seg_length), "w")
        for l in labels_test_checked:
            ltestfile.write(str(l) + "\n")
        ltestfile.close()


        if val:
            np.savetxt(froot+"%i_features_val.txt"%int(seg_length), features_val_checked["features"])
            np.savetxt(froot+"%i_tstart_val.txt"%int(seg_length), features_val_checked["tstart"])

            lvalfile = open(froot+"%i_labels_val.txt"%int(seg_length), "w")
            for l in labels_val_checked:
                lvalfile.write(str(l) + "\n")
            lvalfile.close()


        if lc:
            lc_all = {"train":features_train["lc"], "test":features_test["lc"]}
            if val:
                lc_all["val"] = features_val["lc"]

            f = open(froot+"%i_lc_all.dat"%int(seg_length), "w")
            pickle.dump(lc_all, f, -1)
            f.close()

        if hr:
            hr_all = {"train":features_train["hr"], "test":features_test["hr"]}
            if val:
                hr_all["val"] = features_val["hr"]

            f = open(froot+"%i_hr_all.dat"%int(seg_length), "w")
            pickle.dump(hr_all, f, -1)
            f.close()
            
    return labelled_features



## Using the Script

For the actual analysis, I have put the stuff above into a script `feature_extraction.py`. 
We will run it on segments with the following choices determined by cross validation and 
common sense (see also feature engineering notebook):

1. segments are 1024 seconds long (for supervised classification) and 256 seconds for unsupervised classification
2. the overlap between consecutive segments is 64 seconds
3. the fraction of training data is 0.5, those of validation and test data 0.25
4. the number of weights, $k=$ for the linear filter (cross validation)
5. the regularization parameter for ridge regression in the linear filter, is $\alpha=$ (cross validation)

More to come!

3 (looks good, but samples lost when removing NaN values!)
6 (also looses samples during removing NaNs)
20160516 (low val score)
20160523 
20160525 (low val score)
20160528 (low val score)
20160601 (doesn't match results)
20160606 (so far most decent, but still mis-classifies omega state)
20160607 (best so far; manages to classify all)
20160608 (does not work!)

In [120]:
reload(feature_extraction)

feature_extraction.extract_all(d_all, seg_length_all=[1024.], overlap=[256.],
                val=True, train_frac=0.5, validation_frac = 0.25, test_frac = 0.25,
                datadir="../../", seed=20160615)

feature_extraction.py:128: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  segments.append(dtemp)


The number of states in the training set is 17
The number of states in the test set is 17
The number of states in the validation set is 17


KeyboardInterrupt: 